In [9]:
# yasserh/titanic-dataset
import pandas as pd


file_path = "../data/Titanic-Dataset.csv"

try:
    df = pd.read_csv(file_path)
except FileNotFoundError:
    print("error opening file")
    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [10]:
most_frequent_embarked = df['Embarked'].mode()[0]
print(f"! - {most_frequent_embarked}")

# inplace=True (изменения прям к df)
# df['Embarked'].fillna(most_frequent_embarked, inplace=True)

df.fillna({'Embarked': most_frequent_embarked}, inplace=True)

df.info()

! - S
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [11]:
median_age = df['Age'].median()
print(f"! - {median_age}")

# старый (лучше отказаться)
# df['Age'].fillna(median_age, inplace=True)
# новый
df.fillna({'Age': median_age}, inplace=True)

df.info()

! - 28.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [12]:
sex_mapping = {'male' : 0, 'female' : 1}

df['Sex'] = df['Sex'].map(sex_mapping)

df['Has_Cabin'] = df['Cabin'].notna().astype(int)

df.drop(['Cabin', 'Name', 'Ticket'], axis=1, inplace=True)

df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    int64  
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Has_Cabin    891 non-null    int64  
 9   Embarked_Q   891 non-null    bool   
 10  Embarked_S   891 non-null    bool   
dtypes: bool(2), float64(2), int64(7)
memory usage: 64.5 KB


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Has_Cabin,Embarked_Q,Embarked_S
0,1,0,3,0,22.0,1,0,7.2500,0,False,True
1,2,1,1,1,38.0,1,0,71.2833,1,False,False
2,3,1,3,1,26.0,0,0,7.9250,0,False,True
3,4,1,1,1,35.0,1,0,53.1000,1,False,True
4,5,0,3,0,35.0,0,0,8.0500,0,False,True


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X = df.drop(['Survived', 'PassengerId'], axis=1)
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=52)

model = LogisticRegression(random_state=52, max_iter=250)
 
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f"{accuracy:.4f}")


0.7989


In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model_scaled = LogisticRegression(random_state=52, max_iter=250)

model_scaled.fit(X_train_scaled, y_train)

y_pred_scaled = model_scaled.predict(X_test_scaled)

accuracy_scaled = accuracy_score(y_test, y_pred_scaled)

print(f"{accuracy_scaled:.4f}")

coef_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': model.coef_[0]
}).sort_values(by='Coefficient', ascending=False)

print(coef_df)

0.7989
      Feature  Coefficient
1         Sex     2.596167
6   Has_Cabin     0.562402
5        Fare     0.001813
2         Age    -0.037179
7  Embarked_Q    -0.074407
4       Parch    -0.133071
3       SibSp    -0.275321
8  Embarked_S    -0.341599
0      Pclass    -0.913306


In [15]:
age_bins = [0, 16, 32, 48, 64, 100]
age_labels = ['Child', 'Young Adult', 'Middle Age', 'Senior', 'Elderly']

df['AgeGroup'] = pd.cut(df['Age'], bins=age_bins, labels=age_labels, right=False)

df = pd.get_dummies(df, columns=['AgeGroup'], prefix='', drop_first=True)
df.drop('Age', axis=1, inplace=True) 


X = df.drop(['Survived', 'PassengerId'], axis=1)
y = df['Survived']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=52
)


scaler = StandardScaler()


X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


model = LogisticRegression(random_state=42, max_iter=100)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)

print(f"{accuracy:.4f}")

coef_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': model.coef_[0]
}).sort_values(by='Coefficient', ascending=False)

print(coef_df)



0.7989
         Feature  Coefficient
1            Sex     1.281514
5      Has_Cabin     0.221141
4           Fare     0.123308
6     Embarked_Q    -0.022143
7     Embarked_S    -0.156612
3          Parch    -0.160822
2          SibSp    -0.358008
11      _Elderly    -0.387170
10       _Senior    -0.476057
9    _Middle Age    -0.564644
8   _Young Adult    -0.666071
0         Pclass    -0.692862


In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_model = RandomForestClassifier(n_estimators=420, random_state=42, min_samples_split=2, max_depth=6)
rf_model.fit(X_train, y_train) 

rf_y_pred = rf_model.predict(X_test)

rf_accuracy = accuracy_score(y_test, rf_y_pred)

print(f"{rf_accuracy:.4f}")

0.8324
